# Predicting Insurance Claims Fraud

Insurance claim fraud can be very costly for insurance companies. Automatically identifying suspicious elements of a claim can help predict fraud in a quick and efficient manner. 

This demo will show you an end to end Watson Studio flow written in Python to identify potential fraud in insurance claims. This will include accessing a DB2 instance, data processing, training a model, and deploying the model. 

   1. Install packages
   2. Load data
       - Import data from Data Catalog (Insurance Claims, Insured Drivers, Insurance Policies)
       - Insert data as dataframes 
       - Determine relevant columns 
   3. Prepare data
       - Join dataframes
       - Create feature columns
   4. Train model
   5. Deploy model


## 1. Install packages

In [1]:
!pip install scikit-learn

In [2]:
import sklearn
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.io import arff
import brunel
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2020-01-28 11:13:07,908 - watson_machine_learning_client.wml_client_error - WARNING - Error during getting ML Token. (GET https://ibm-watson-ml.mybluemix.net/v3/identity/token)
Status code: 400, body: {"message": "Use IAM token as ML token was discontinued"}


## 2. Load data

This portion of the notebook will show you how to load data from a DB2 Database. It will also slim down the data to only the relevant data. This is pulling data from three sources: Insurance Claims, Insured Drivers, Insurance Policies.

##### Insurance Claims

In [3]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_bdce8ba7fd244ec3819d04394d0beedf = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vl-4cAfXIXDAjSxFHQdwPQJo66ntjrP2ziTT67LOsoqg',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_bdce8ba7fd244ec3819d04394d0beedf.get_object(Bucket='datasciencedemo-donotdelete-pr-t5fotxtat1jued',Key='INSURANCE_CLAIM.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,HOUSEHOLD_ID,DRIVER_ID,POLICY_ID,CLAIM_ID,INCIDENT_CAUSE,DESCRIPTION,CLAIM_STATUS,ODOMETER_AT_LOSS,LOSS_EVENT_TIME,CLAIM_INIT_TIME,POLICE_REPORT,CLAIMS_AT_LOSS_DATE,LOSS_LOCATION_LAT,LOSS_LOCATION_LONG,CLAIM_AMOUNT,FLAG_FOR_FRAUD_INV
0,EH29521,XDY4674,US7684354,A-2017-QK123,2,NaN,3,234647.4,2017-02-20 00:00:00.0,2017-03-04 00:00:00.0,1,1,41.816358,-87.621548,1190.0,0
1,MI69314,OTR6593,VP9755242,A-2017-UK576,1,NaN,3,216841.1,2017-02-23 00:00:00.0,2017-02-24 00:00:00.0,1,1,41.709442,-87.720887,3208.5,0
2,ME64244,ZDX5513,TU9017020,A-2017-XY722,3,NaN,3,259674.7,2017-02-25 00:00:00.0,2017-03-03 00:00:00.0,0,1,41.771133,-87.571692,2560.0,0
3,GE41016,JQW6902,ZT8009136,A-2017-WI822,4,NaN,3,185866.0,2017-02-26 00:00:00.0,2017-03-05 00:00:00.0,0,1,42.010314,-87.662608,1380.0,0
4,II25056,CVI8835,OJ5466096,A-2017-YI133,4,NaN,3,109080.0,2017-03-01 00:00:00.0,2017-03-13 00:00:00.0,0,3,41.880781,-87.730232,1230.5,0


In [6]:
%brunel data("df_data_1") x(FLAG_FOR_FRAUD_INV) y(CLAIM_AMOUNT)

<IPython.core.display.Javascript object>

In [7]:
%brunel data("df_data_1") y(CLAIMS_AT_LOSS_DATE) x(FLAG_FOR_FRAUD_INV)

<IPython.core.display.Javascript object>

In [8]:
%brunel data("df_data_1") x(FLAG_FOR_FRAUD_INV) y(ODOMETER_AT_LOSS)

<IPython.core.display.Javascript object>

In [9]:
%brunel data("df_data_1") x(FLAG_FOR_FRAUD_INV) y(POLICE_REPORT)

<IPython.core.display.Javascript object>

In [10]:
rel_cols = ['HOUSEHOLD_ID','DRIVER_ID','POLICY_ID','CLAIM_ID','LOSS_EVENT_TIME','CLAIM_INIT_TIME', 'ODOMETER_AT_LOSS',
 'CLAIMS_AT_LOSS_DATE','LOSS_LOCATION_LAT','LOSS_LOCATION_LONG','CLAIM_AMOUNT','FLAG_FOR_FRAUD_INV']

In [11]:
df_claim = df_data_1[rel_cols]

##### Insured drivers

In [12]:

body = client_bdce8ba7fd244ec3819d04394d0beedf.get_object(Bucket='datasciencedemo-donotdelete-pr-t5fotxtat1jued',Key='INSURANCE_DRIVER.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()


,HOUSEHOLD_ID,DRIVER_ID,FIRST_NAME,LAST_NAME,GENDER,BIRTHDATE,SSN,DRIVERS_LICENSE_ID,DRIVERS_LICENSE_EXPIRY,DRIVERS_LICENSE_STATE,...,WORK_ADDR_NUMB,WORK_ADDR_STREET_NAME,WORK_ADDR_UNIT_DESIGNATION,WORK_ADDR_UNIT_NUMBER,WORK_ADDR_CITY,WORK_ADDR_STATE,WORK_ADDR_ZIPCODE,WORK_ADDR_LATITUDE,WORK_ADDR_LONGITUDE,WORK_ADDR_PBKEY
0,HE13983,JYE4178,Nan,Ochs,F,1947-10-10,843-58-5890,L246-8795-3227,2020-04-28,IL,...,660,HILL DR,APT,5-207,HOFFMAN ESTATES,IL,60169,42.046272,-88.096735,P00007UWK06D
1,JE03959,FMO6665,Samual,Rasnick,M,1972-03-07,158-11-2748,N117-7321-2332,2018-08-27,IL,...,8564,W LAWRENCE AVE,NaN,NaN,NORRIDGE,IL,60706,41.966851,-87.841139,P00007UWGKL1
2,JE03959,DEC7791,Nelly,Rasnick,F,1963-11-23,135-93-5519,V280-2176-1946,2018-06-04,IL,...,1935,S WABASH AVE,APT,P119,CHICAGO,IL,60616,41.855900,-87.625075,P00007UWODP3
3,KE49783,EDG6737,Gail,Bonora,F,1985-06-05,444-47-7623,T372-8252-1710,2019-03-09,IL,...,2708,HEBRON AVE,NaN,NaN,ZION,IL,60099,42.445409,-87.846973,P00007UWC3PJ
4,KE49783,FMF3124,Abe,Bonora,M,1949-08-06,427-64-4017,H804-4283-8450,2019-08-03,IL,...,8428,S CONSTANCE AVE,NaN,NaN,CHICAGO,IL,60617,41.741584,-87.579785,P00007UUH1GM


In [13]:
rel_cols_2 = ["DRIVER_ID","DRIVERS_LICENSE_EXPIRY","DATE_AT_CURRENT_ADDRESS"]

In [15]:
df_driver = df_data_2[rel_cols_2]

##### Insurance policy

In [16]:

body = client_bdce8ba7fd244ec3819d04394d0beedf.get_object(Bucket='datasciencedemo-donotdelete-pr-t5fotxtat1jued',Key='INSURANCE_POLICY.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()


,HOUSEHOLD_ID,PRIMARY_DRIVER_ID,POLICY_ID,START_DATE,EXPIRY_DATE,MODEL_YEAR,MAKE,MODEL,PLATE,COLOR,INITIAL_ODOMETER,LOW_MILEAGE_USE,CREDIT_CARD_TYPE,CREDIT_CARD_NUMBER,CREDIT_CARD_CCV,CREDIT_CARD_EXPIRY
0,ID23657,FAJ7963,PY3974800,2017-10-23,2018-10-23,1995,Ford,Expedition,LSFZ630,Light Brown,526332.0,0,VISA,4470482619317178,765,2020-02-01
1,GJ57163,NFR7733,US0312194,2015-10-19,2016-10-18,2005,Chevrolet,Colorado,MKFS507,Light Brown,212256.0,0,VISA,4705313989935330,423,2020-01-01
2,FG70555,BMI4402,VP5674769,2015-06-13,2016-06-12,2003,Ford,Transit,EDCD682,White,243672.0,0,MC,5336910571987522,49,2022-04-01
3,FG70555,BMI4402,YO1286547,2015-04-08,2016-04-07,2013,Ford,Focus,RURZ148,Med Dk Blue,30780.0,1,MC,5336910571987522,49,2022-04-01
4,FG70555,BMI4402,VU3750483,2016-10-16,2017-10-16,2003,Ford,Transit,EDCD682,White,266832.0,0,MC,5336910571987522,49,2022-04-01


In [17]:
rel_cols_3 = ["POLICY_ID","EXPIRY_DATE","START_DATE","LOW_MILEAGE_USE"]

In [18]:
df_policy = df_data_3[rel_cols_3]

## 3. Prepare Data

This section of the notebook will show you how to prepare the seperate dataframes to create one dataframe that will be used for training a model. This section will also do some data manipulation to create features for the dataframe

### Join dataframes

In [19]:
df_f = pd.merge(df_claim, df_driver, on='DRIVER_ID', how='left')

df_f = pd.merge(df_f,df_policy, on = 'POLICY_ID', how = 'inner')

In [20]:
df_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 0 to 974
Data columns (total 17 columns):
HOUSEHOLD_ID               975 non-null object
DRIVER_ID                  975 non-null object
POLICY_ID                  975 non-null object
CLAIM_ID                   975 non-null object
LOSS_EVENT_TIME            975 non-null object
CLAIM_INIT_TIME            975 non-null object
ODOMETER_AT_LOSS           975 non-null float64
CLAIMS_AT_LOSS_DATE        975 non-null int64
LOSS_LOCATION_LAT          975 non-null float64
LOSS_LOCATION_LONG         975 non-null float64
CLAIM_AMOUNT               975 non-null float64
FLAG_FOR_FRAUD_INV         975 non-null int64
DRIVERS_LICENSE_EXPIRY     975 non-null object
DATE_AT_CURRENT_ADDRESS    975 non-null object
EXPIRY_DATE                975 non-null object
START_DATE                 975 non-null object
LOW_MILEAGE_USE            975 non-null int64
dtypes: float64(4), int64(3), object(10)
memory usage: 137.1+ KB


### Identify and Create Feature Columns

##### Claim filed within 15 days of policy expiry

In [23]:
# Claim within 15 days of policy expiry (date of loss - insurance_policy.expiry)
df_f["EXPIRY_DATE"] = pd.to_datetime(df_f["EXPIRY_DATE"])
df_f["LOSS_EVENT_TIME"] = pd.to_datetime(df_f["LOSS_EVENT_TIME"])

df_f["DAYS_FROM_LOSS"] = df_f["LOSS_EVENT_TIME"] - df_f["EXPIRY_DATE"]
df_f["DAYS_FROM_LOSS"] = abs(df_f.DAYS_FROM_LOSS.dt.days)

df_f.loc[df_f['DAYS_FROM_LOSS'] >= 15, 'SUSPICIOUS_CLAIM_TIME'] = 1
df_f.loc[df_f['DAYS_FROM_LOSS'] < 15, 'SUSPICIOUS_CLAIM_TIME'] = 0

In [24]:
df_f["SUSPICIOUS_CLAIM_TIME"].value_counts()

1.0    915
0.0     60
Name: SUSPICIOUS_CLAIM_TIME, dtype: int64

##### Expired drivers license


In [25]:
# Expired drivers license (if date of loss > insurance_driver.drivers_license_expiry)
df_f["DRIVERS_LICENSE_EXPIRY"] = pd.to_datetime(df_f["DRIVERS_LICENSE_EXPIRY"])


df_f["DAYS_FROM_L_EXPIRY"] = df_f["DRIVERS_LICENSE_EXPIRY"] - df_f["LOSS_EVENT_TIME"]
df_f["DAYS_FROM_L_EXPIRY"] = df_f.DAYS_FROM_L_EXPIRY.dt.days

df_f.loc[df_f['DAYS_FROM_L_EXPIRY'] >= 0, 'EXPIRED_LICENSE'] = 0
df_f.loc[df_f['DAYS_FROM_L_EXPIRY'] < 0, 'EXPIRED_LICENSE'] = 1


##### Claim filed within 15 days of moving

In [26]:
# Days living at current address (date of loss - insurance_driver.date_at_current_address)

df_f["DATE_AT_CURRENT_ADDRESS"] = pd.to_datetime(df_f["DATE_AT_CURRENT_ADDRESS"])


df_f["DAYS_AT_ADDRESS"] = df_f["LOSS_EVENT_TIME"] - df_f["DATE_AT_CURRENT_ADDRESS"]
df_f["DAYS_AT_ADDRESS"] = abs(df_f.DAYS_AT_ADDRESS.dt.days)

df_f.loc[df_f['DAYS_AT_ADDRESS'] >= 15, 'SUSPICIOUS_LIVING'] = 1
df_f.loc[df_f['DAYS_AT_ADDRESS'] < 15, 'SUSPICIOUS_LIVING'] = 0

##### Client with a low mileage discount not having low mileage

In [27]:
#7500/year
df_f["START_DATE"] = pd.to_datetime(df_f["START_DATE"])
#find number of days between policy creation and accident
df_f["LENGTH_OF_POLICY"]=(df_f["LOSS_EVENT_TIME"] - df_f["START_DATE"]).dt.days

#convert to years
df_f["LENGTH_OF_POLICY"]=df_f["LENGTH_OF_POLICY"]/365

#divide Odometer at loss by years
df_f["MILES/YEAR"] = df_f["ODOMETER_AT_LOSS"]/df_f["LENGTH_OF_POLICY"]
df_f["MILES/YEAR"].value_counts()



145432.118234    1
234953.253233    1
311332.643229    1
55515.574648     1
123525.401515    1
145305.160550    1
41161.635017     1
316044.542945    1
270859.128079    1
343652.573991    1
357034.135714    1
444934.671171    1
47761.055147     1
480395.298283    1
97407.709325     1
83715.343421     1
836054.660714    1
225418.465704    1
49174.210227     1
333973.819652    1
87012.862810     1
138899.077083    1
55960.766821     1
127652.911633    1
158134.051821    1
148190.243875    1
128341.984375    1
388218.025735    1
38327.581707     1
198129.673295    1
                ..
138485.866667    1
43028.581560     1
219977.172330    1
205708.160000    1
227492.647193    1
44534.372396     1
378668.921594    1
213403.812808    1
129338.850177    1
433650.444700    1
145754.334539    1
102579.963602    1
924887.694444    1
258935.458015    1
206160.525547    1
110428.874687    1
355749.475610    1
140508.777778    1
189399.690217    1
54898.636569     1
42573.963602     1
153467.34502

In [28]:

# Conflict on whether a policyholder with a low mileage discount experienced a loss with high mileage at the point of loss
df_f.loc[df_f["MILES/YEAR"] <7500, 'LOW_MILEAGE_AT_LOSS'] = 1
df_f.loc[df_f["MILES/YEAR"] >=7500, 'LOW_MILEAGE_AT_LOSS'] = 0

In [29]:
df_f.loc[df_f["LOW_MILEAGE_USE"]==df_f["LOW_MILEAGE_AT_LOSS"], 'SUSPICIOUS_MILEAGE'] = 0
df_f.loc[df_f["LOW_MILEAGE_USE"]!= df_f["LOW_MILEAGE_AT_LOSS"], 'SUSPICIOUS_MILEAGE'] = 1

##### Excessive (over $3000) Claim Amount

In [30]:
df_f.loc[df_f["CLAIM_AMOUNT"] <3000, 'EXCESSIVE_CLAIM_AMOUNT'] = 0
df_f.loc[df_f["CLAIM_AMOUNT"] >=3000, 'EXCESSIVE_CLAIM_AMOUNT'] = 1

### Create dataframe of certain features

In [31]:
features = ['FLAG_FOR_FRAUD_INV', 
 'SUSPICIOUS_MILEAGE',
 'EXPIRED_LICENSE',
 'SUSPICIOUS_CLAIM_TIME',
 'SUSPICIOUS_LIVING',
 'EXCESSIVE_CLAIM_AMOUNT']

In [32]:
df_model = df_f[features]

In [33]:
#ensure all relevant features are integers
df_model["SUSPICIOUS_LIVING"] = df_model["SUSPICIOUS_LIVING"].astype(int)
df_model["EXPIRED_LICENSE"] = df_model["EXPIRED_LICENSE"].astype(int)
df_model["SUSPICIOUS_CLAIM_TIME"] = df_model["SUSPICIOUS_CLAIM_TIME"].astype(int)
df_model["SUSPICIOUS_MILEAGE"] = df_model["SUSPICIOUS_MILEAGE"].astype(int)
df_model["EXCESSIVE_CLAIM_AMOUNT"] = df_model["EXCESSIVE_CLAIM_AMOUNT"].astype(int)

##### Overall Look at Features

In [34]:
df_f.groupby("FLAG_FOR_FRAUD_INV", as_index=False).mean()

,FLAG_FOR_FRAUD_INV,ODOMETER_AT_LOSS,CLAIMS_AT_LOSS_DATE,LOSS_LOCATION_LAT,LOSS_LOCATION_LONG,CLAIM_AMOUNT,LOW_MILEAGE_USE,DAYS_FROM_LOSS,SUSPICIOUS_CLAIM_TIME,DAYS_FROM_L_EXPIRY,EXPIRED_LICENSE,DAYS_AT_ADDRESS,SUSPICIOUS_LIVING,LENGTH_OF_POLICY,MILES/YEAR,LOW_MILEAGE_AT_LOSS,SUSPICIOUS_MILEAGE,EXCESSIVE_CLAIM_AMOUNT
0,0,180783.846046,1.492091,41.857895,-87.675014,2521.147627,0.094903,117.021090,0.933216,894.546573,0.042179,3487.956063,0.996485,1.070159,205170.356701,0.0,0.094903,0.086116
1,1,175504.046305,1.758621,41.860133,-87.679617,26429.480296,0.088670,118.160099,0.945813,917.960591,0.036946,3252.413793,1.000000,0.960490,230687.578997,0.0,0.088670,0.933498


## 4. Train model

In [35]:
#split data into x and y variables
xVar = df_model[["EXPIRED_LICENSE","SUSPICIOUS_CLAIM_TIME","SUSPICIOUS_LIVING","SUSPICIOUS_MILEAGE","EXCESSIVE_CLAIM_AMOUNT"]]
yVar = df_model["FLAG_FOR_FRAUD_INV"]

In [36]:
xVar.head()

,EXPIRED_LICENSE,SUSPICIOUS_CLAIM_TIME,SUSPICIOUS_LIVING,SUSPICIOUS_MILEAGE,EXCESSIVE_CLAIM_AMOUNT
0,0,1,1,0,0
1,0,1,1,0,1
2,0,1,1,0,0
3,0,1,1,0,0
4,0,1,1,0,0


In [37]:
#split into a test/train set
X_train, X_test, y_train, y_test = train_test_split(xVar, yVar, test_size=0.2)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)


(780, 5) (780,)
(195, 5) (195,)


In [38]:
#train model
clf = RandomForestClassifier(n_jobs=2, random_state=0)

clf.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [39]:
#create confusion matrix to gut check model
preds = clf.predict(X_test)
pd.crosstab(y_test, preds, rownames=['Actual Result'], colnames=['Predicted Result'])

Predicted Result,0,1
Actual Result,,
0,110,12
1,4,69
